## NeuroFinder Processing Tool - Jupyter Notebook Version

Welcome to the NeuroFinder Processing Tool. This Jupyter Notebook provides a non-GUI interface to run the project and get familiar with its functionalities. You can process your data files, update databases, and generate reports directly within this notebook.

The NeuroFinder Processing Tool automates the management of a comprehensive database containing company information related to neurotechnology. It facilitates the import, standardization, validation, and updating of company data files in multiple formats (e.g., CSV, Excel).

### Objective of This Notebook

This notebook aims to:
* Provide an interactive environment to run the NeuroFinder Processing Tool without the GUI.
* Allow you to load data files, process them, and export the results.
* Help you get familiar with the tool's functionalities.

### Prerequisites

Before running this notebook, ensure you have:

* Python 3.x installed.
* Necessary Python packages (we will install them in the next step).
* Access to the data files you wish to process.
* The main database files (main_database.xlsx, not_neurotech_database.xlsx).

In [ ]:
# Install required packages
!pip install pandas openpyxl requests python-dotenv matplotlib seaborn sqlite3


In [ ]:
!python.exe -m pip install --upgrade pip


In [2]:
# Import standard libraries
import os
import re
import unicodedata
from datetime import datetime as dt

# Import third-party libraries
import pandas as pd
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


### Loading Environment Variables

If you have a .env file with environment variables, you can load it using python-dotenv. Otherwise, we can set default paths.

In [3]:
# Load environment variables
load_dotenv()
MAIN_DB_PATH = os.getenv('MAIN_DB_PATH')
NOT_NEUROTECH_DB_PATH = os.getenv('NOT_NEUROTECH_DB_PATH')
NEW_COMPANIES_PATH = os.getenv('NEW_COMPANIES_PATH')
UPDATED_COMPANIES_PATH = os.getenv('UPDATED_COMPANIES_PATH')


Defining Helper Functions

In [4]:
def clean_value(value):
    """Cleans the input value by stripping unwanted characters and converting to int if possible."""
    if pd.isna(value):
        return value
    cleaned_value = str(value).strip('="')
    try:
        return int(cleaned_value)
    except ValueError:
        return cleaned_value

def clean_dataframe(filepath, file_type='csv'):
    """Reads a file into a DataFrame, cleans it, and returns the cleaned DataFrame."""
    read_function = pd.read_csv if file_type == 'csv' else pd.read_excel
    df = read_function(filepath, index_col=False,
                       engine='openpyxl' if file_type == 'excel' else None)
    if 'former company names' in df.columns:
        df['former company names'] = df['former company names'].astype(str)
    for col in df.columns:
        df[col] = df[col].apply(clean_value)
    return df

def escape_special_characters(name: str) -> str:
    """Replaces special characters in a filename with underscores to ensure compatibility."""
    return re.sub(r'[^a-zA-Z0-9-_]', '_', name)


### Initializing the Database Handler

Create an instance of the DbHandler class to manage your databases.

In [5]:
from main.backend import DbHandler
# Initialize the database handler
db_handler = DbHandler(MAIN_DB_PATH, NOT_NEUROTECH_DB_PATH)

# Review the data

In [6]:
db_handler.main_db.describe()

,Company Founded Year,Last Funding Amount,Total Funding Amount,Number of Funding Rounds,Company Number of Investors,Company Number of Investments,acquired,Inactive Year,Number of Patents,Unnamed: 53,Contact Name
count,262.000000,9.300000e+01,1.080000e+02,124.000000,116.000000,6.000000,95.000000,54.000000,1.0,0.0,0.0
mean,2013.454198,9.096734e+06,2.579617e+07,2.548387,3.939655,3.000000,0.105263,2019.222222,13.0,NaN,NaN
std,11.614324,1.858834e+07,6.910025e+07,2.123637,4.175058,1.264911,0.308520,3.451369,NaN,NaN,NaN
min,1905.000000,1.000000e+04,1.600000e+04,0.000000,0.000000,1.000000,0.000000,2007.000000,13.0,NaN,NaN
25%,2011.000000,8.500000e+05,1.310000e+06,1.000000,1.000000,2.250000,0.000000,2017.000000,13.0,NaN,NaN
50%,2016.000000,2.200000e+06,4.000000e+06,2.000000,2.000000,3.500000,0.000000,2019.000000,13.0,NaN,NaN
75%,2019.000000,1.000000e+07,2.211750e+07,3.000000,5.000000,4.000000,0.000000,2022.000000,13.0,NaN,NaN
max,2024.000000,1.500000e+08,5.569000e+08,10.000000,20.000000,4.000000,1.000000,2024.000000,13.0,NaN,NaN


In [7]:
db_handler.main_db.shape
# 659 companies X 58 columns (features) in the main database

(273, 60)

In [9]:
db_handler.main_db.head()

,Company Name,Updating_Date,Logo in Visualization folder?,"Operation Status (Active=True, False = False)",INCLUSION,Operation/relevant Notes,Website,Startup Nation Page,Neurotech Category,Market Category,...,Inactive Year,Number of Patents,Comments,Unnamed: 53,Contact Name,Contact Phone Number / Email,האם יצרנו איתם כבר קשר? (כדי לא להתיש),BrainstormIL contact,Unnamed: 58,Unnamed: 59
0,AcousticView,2024-02-14 00:00:00,yes,True,True,NaN,http://www.acousticview.com/,https://finder.startupnationcentral.org/compan...,Imaging | Neuromonitoring,Medical devices | Medical equipment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ActualSignal,2024-07-14 00:00:00,No,True,True,NaN,https://www.actualsignal.com/,https://finder.startupnationcentral.org/compan...,NeuroreHabilitation | NeuroDegenerative | Neur...,Digital & Health care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam CogTech,2024-02-14 00:00:00,yes,True,True,NaN,http://adam-cogtec.com/,https://finder.startupnationcentral.org/compan...,Cognitive Assessment & Enhancement,Consumer Electronics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,אסף הראל,NaN,NaN
3,AlgoSensus,2024-02-14 00:00:00,yes,True,True,NaN,https://www.algosensus.com/,https://finder.startupnationcentral.org/compan...,Cognitive Assessment & Enhancement,Medical devices | Medical equipment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alpha Omega,2024-02-14 00:00:00,yes,True,True,NaN,http://www.alphaomega-eng.com,https://finder.startupnationcentral.org/compan...,NeuroSurgery | NeuroDevices,Medical devices | Medical equipment,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Review functions

#### search new companies

In [10]:
# Lets check the new copmanies database shape: 0 compnaies = the file is empty
db_handler.new_companies_db.shape

(0, 60)

In [20]:
# Let's view the new potenital companies from CrunchBase
cb_path = "jan25/crunchbase search.csv"
cb_new_data = clean_dataframe(cb_path)
cb_new_data.head() 

,Organization Name,Organization Name URL,Founded Date,Founded Date Precision,Full Description,Industries,Headquarters Location,Description,CB Rank (Company)
0,Firefly Neuroscience,https://www.crunchbase.com/organization/firefl...,2006-01-01,year,"Firefly Neuroscience utilizes big data, signal...","Artificial Intelligence (AI), Big Data, Health...","Herzliya, Tel Aviv, Israel","Firefly Neuroscience utilizes big data, signal...","1,117"
1,BioCatch,https://www.crunchbase.com/organization/biocatch,2011-01-01,year,BioCatch is the leader in Behavioral Biometric...,"Analytics, Cyber Security, FinTech, Fraud Dete...","Tel Aviv, Tel Aviv, Israel",BioCatch unlocks the power of behavior and del...,"1,419"
2,Wearable Devices,https://www.crunchbase.com/organization/wearab...,2014-03-13,day,Wearable Devices Ltd. (NASDAQ: WLDS) is a grow...,"Artificial Intelligence (AI), Augmented Realit...","Yoqne`am `illit, HaZafon, Israel","Developing Mudra, a Brain-Computer Interface ...","1,428"
3,NeuroSense Therapeutics,https://www.crunchbase.com/organization/neuros...,2017-01-01,year,"NeuroSense Therapeutics Ltd., is a clinical-st...","Biotechnology, Neuroscience, Therapeutics","Herzliya, Tel Aviv, Israel",NeuroSense Therapeutics is a clinical-stage dr...,"2,391"
4,RailVision,https://www.crunchbase.com/organization/railvi...,2015-01-01,year,Rail Vision is focusing on train safety securi...,"Automotive, Big Data, Image Recognition, Secur...","Ra'anana, HaMerkaz, Israel",RailVision introduces an Automated Early Warni...,"4,513"


In [21]:
# Lets check the new crunchbase data shape
cb_new_data.shape

(265, 9)

In [22]:
# Let's start the search prcoess with the crunchbase file path and data_type as "cb"
db_handler.start_searching_process(file_path=cb_path, data_type="cb")
db_handler.new_companies_db.shape # Lets check the new copmanies data base shape

(24, 61)

#### Update new copmanies

In [23]:
db_handler.update_companies_db.shape

(0, 60)

In [25]:
# Let's view the new potenital companies from The Start Up Nation Central
tsun_path = 'jan25/brain1.csv'
tsun_new_data = clean_dataframe(tsun_path)
tsun_new_data.head() 

,Name,Finder URL,Description,Primary Sector,Founded,Employees,Funding Stage
0,BRAIN.Q,https://finder.startupnationcentral.org/compan...,Restorative Brain Health Therapeutics to Rever...,Health Tech & Life Sciences,2016,11-50,B
1,brain.space,https://finder.startupnationcentral.org/compan...,The Brain Data Company,Data Analysis & Decision Support,2018,11-50,A
2,Brain1,https://finder.startupnationcentral.org/compan...,Brainwave Entrainment Device,Health Tech & Life Sciences,2015,1-10,Pre-Funding
3,BrainBalance,https://finder.startupnationcentral.org/compan...,Minimally Invasive Device for the Treatment of...,Health Tech & Life Sciences,2016,1-10,Pre-Funding
4,BrainCommerce,https://finder.startupnationcentral.org/compan...,AI Sales Agent for E-Commerce,Business Software,2024,1-10,Pre-Funding


In [26]:
# Lets check the new  data shape
tsun_new_data.shape

(50, 7)

In [27]:
# Let's start the search prcoess with the crunchbase file path and data_type as "cb"
db_handler.start_searching_process(file_path=tsun_path, data_type="tsun")
db_handler.new_companies_db.shape

(31, 61)

In [28]:
db_handler.update_companies_db.shape

(0, 60)

In [30]:
df = pd.read_excel('main/data/not_neurotech.xlsx')

In [31]:
df.head()

,Company Name,Updating_Date,Logo in Visualization folder?,"Operation Status (Active=True, False = False)",INCLUSION,Operation/relevant Notes,Website,Startup Nation Page,Neurotech_Category,Market_Category,...,acquired,product_stage,Number of Patents,Comments,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57
0,1E Therapeutics,2024-02-14 00:00:00,yes,1.0,True,Not only neurotech,https://www.1etx.com/,https://finder.startupnationcentral.org/compan...,NeuroPharmacology | NeuroBioTechnology,Biotechnology & Biopharmaceutical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2breathe Technologies Ltd,2023-06-08 00:00:00,yes,0.0,False,Not relevant to Neurotech/IL,2breathe.com/about-us/,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4Girls,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A-muse,NaN,NaN,NaN,False,INACTIVE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AbiliSense,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.describe()

,"Operation Status (Active=True, False = False)",Company_Founded_Year,Total_Funding_Amount,Total_Funding_Amount_M_dollars,Number_of_Funding_Rounds,Company_Number_of_Investors,Company_Number_of_Investments,Number of Patents,Unnamed: 53,Unnamed: 54,Unnamed: 55
count,153.000000,230.000000,4.400000e+01,32.000000,48.000000,43.000000,3.0,0.0,0.0,0.0,0.0
mean,0.620915,2014.186957,5.693346e+07,24.020813,2.708333,3.860465,2.0,NaN,NaN,NaN,NaN
std,0.486753,14.894529,1.657335e+08,33.632631,2.351625,3.937496,1.0,NaN,NaN,NaN,NaN
min,0.000000,1892.000000,7.142900e+04,0.325000,1.000000,0.000000,1.0,NaN,NaN,NaN,NaN
25%,0.000000,2014.000000,1.000000e+06,1.000000,1.000000,1.000000,1.5,NaN,NaN,NaN,NaN
50%,1.000000,2018.000000,9.300000e+06,4.705000,2.000000,2.000000,2.0,NaN,NaN,NaN,NaN
75%,1.000000,2020.000000,4.187500e+07,36.925000,3.250000,5.500000,2.5,NaN,NaN,NaN,NaN
max,1.000000,2023.000000,9.200000e+08,120.000000,13.000000,16.000000,3.0,NaN,NaN,NaN,NaN


In [33]:
df = pd.read_excel("main/data/NeuroTech Industry IL 2024.xlsx")

In [34]:
df.head()

,Company Name,Updating_Date,Logo in Visualization folder?,"Operation Status (Active=True, False = False)",INCLUSION,Operation/relevant Notes,Website,Startup Nation Page,Neurotech Category,Market Category,...,Inactive Year,Number of Patents,Comments,Unnamed: 53,Contact Name,Contact Phone Number / Email,האם יצרנו איתם כבר קשר? (כדי לא להתיש),BrainstormIL contact,Unnamed: 58,Unnamed: 59
0,AcousticView,2024-02-14 00:00:00,yes,True,True,NaN,http://www.acousticview.com/,https://finder.startupnationcentral.org/compan...,Imaging | Neuromonitoring,Medical devices | Medical equipment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ActualSignal,2024-07-14 00:00:00,No,True,True,NaN,https://www.actualsignal.com/,https://finder.startupnationcentral.org/compan...,NeuroreHabilitation | NeuroDegenerative | Neur...,Digital & Health care,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adam CogTech,2024-02-14 00:00:00,yes,True,True,NaN,http://adam-cogtec.com/,https://finder.startupnationcentral.org/compan...,Cognitive Assessment & Enhancement,Consumer Electronics,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,אסף הראל,NaN,NaN
3,AlgoSensus,2024-02-14 00:00:00,yes,True,True,NaN,https://www.algosensus.com/,https://finder.startupnationcentral.org/compan...,Cognitive Assessment & Enhancement,Medical devices | Medical equipment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alpha Omega,2024-02-14 00:00:00,yes,True,True,NaN,http://www.alphaomega-eng.com,https://finder.startupnationcentral.org/compan...,NeuroSurgery | NeuroDevices,Medical devices | Medical equipment,...,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
col_lst = df.columns
for category in col_lst:
    print(category) 

Company Name
Updating_Date
Logo in Visualization folder?
Operation Status (Active=True, False = False)
INCLUSION
Operation/relevant Notes
Website
Startup Nation Page
Neurotech Category
Market Category
Target Market
TechTools 1
TechTools 2
TechTools 3
Finder Description
Description
Full Description
CB (Crunchbase) Link
Company CB Categories
Company Location
Company Founded Year
Company Number of Employees
Company CB Rank
Funding Status
Last Funding Type
Last Funding Date
Last Funding Amount
Total Funding Amount
Total Funding Amount M dollars
Number of Funding Rounds
Estimated Revenue Range
Company Number of Investors
Company Number of Investments
Company LinkedIn Link
Company LinkedIn Followers Number
Company Contact Email
Company phone number
Founders
CSO
CTO
CEO
COO
CFO
Co-Founder
President
Team Members
Address
former company names
acquired
Product Stage
Inactive Year
Number of Patents
Comments
Unnamed: 53
Contact Name
Contact Phone Number / Email
האם יצרנו איתם כבר קשר? (כדי לא להתיש